### Baseline RandomForest Model: 1) Throw everything in 2) Log features 3) Cross features

In [6]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [2]:
data = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

## Minimum Feature Engineering

In [16]:
# Drop unhelpful cat features
# cat0, cat2, cat4, cat6, cat7 cuz of too much skew
def make_train(data):
    train = data.drop(['cat0', 'cat2', 'cat4', 'cat6', 'cat7'], axis=1)
    
    for col in train.select_dtypes(include='object'):
        le = LabelEncoder()
        train[col] = le.fit_transform(train[col])
    
    return train
#consider removing outlier categories (ex. A nd C for cat5 )
#todo: figure out how to use labelencoder in pipeline

In [17]:
train = make_train(data)

In [20]:
train.shape

(300000, 21)

In [24]:
rf = RandomForestRegressor()
X = train.drop(['id', 'target'], axis=1)
y = train.target

scores = cross_val_score(rf, X, y, cv=5, scoring='neg_root_mean_squared_error', verbose=1, n_jobs=-1)
print(scores.mean)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed: 10.5min remaining: 15.8min


<built-in method mean of numpy.ndarray object at 0x0000020F82BE1350>


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 10.7min finished


In [26]:
scores.mean()

-0.8617550768430446